In [8]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

import pandas as pd

TESTDATA = StringIO("""E;L;Fr;To;P;PI
    E1;L1;FA1;TA7;P1;
    E1;L1;FA1;TA7;P2;Two Way P
    E1;L2;FA8;TA23;P3;
    E1;L2;FA8;TA23;P4;One Way R
    """)

df = pd.read_csv(TESTDATA, sep=";")
df.head()

,E,L,Fr,To,P,PI
0,E1,L1,FA1,TA7,P1,NaN
1,E1,L1,FA1,TA7,P2,Two Way P
2,E1,L2,FA8,TA23,P3,NaN
3,E1,L2,FA8,TA23,P4,One Way R


In [9]:
# cus t
df['key'] = df['E'] + df['L'] + '~~~' + df['Fr'] + '~' + df['To']

In [10]:
# c route
df2 = df[df.PI.notnull()]
df2.head()

,E,L,Fr,To,P,PI,key
1,E1,L1,FA1,TA7,P2,Two Way P,E1L1~~~FA1~TA7
3,E1,L2,FA8,TA23,P4,One Way R,E1L2~~~FA8~TA23


In [11]:
# cus t
df2.rename(columns={'P':'preferredP'}, inplace=True)
df2.head()

/Users/travisjefferies/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,E,L,Fr,To,preferredP,PI,key
1,E1,L1,FA1,TA7,P2,Two Way P,E1L1~~~FA1~TA7
3,E1,L2,FA8,TA23,P4,One Way R,E1L2~~~FA8~TA23


In [12]:
# c route
df3 = df[df.PI.isnull()]
df3.head()

,E,L,Fr,To,P,PI,key
0,E1,L1,FA1,TA7,P1,NaN,E1L1~~~FA1~TA7
2,E1,L2,FA8,TA23,P3,NaN,E1L2~~~FA8~TA23


In [13]:
# r comb
result = df2.merge(df3[['P', 'key']], on='key')

In [15]:
result = result[['E', 'L', 'key', 'preferredP', 'P', 'PI']]
result

,E,L,key,preferredP,P,PI
0,E1,L1,E1L1~~~FA1~TA7,P2,P1,Two Way P
1,E1,L2,E1L2~~~FA8~TA23,P4,P3,One Way R


In [24]:
# cus t
from_and_to = result['key'].str.split('~~~',expand=True)[1].str.split('~', expand=True)
from_and_to.rename(columns={0: 'Fr', 1: 'To'}, inplace=True)
from_and_to

,Fr,To
0,FA1,TA7
1,FA8,TA23


In [27]:
# r comb
result[['E', 'L', 'preferredP', 'P']].merge(from_and_to, left_index=True, right_index=True)

,E,L,preferredP,P,Fr,To
0,E1,L1,P2,P1,FA1,TA7
1,E1,L2,P4,P3,FA8,TA23
